In [ ]:
!pip install torch torchvision timm pandas numpy pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/Trabajo Final/train_df.csv" /content/
!cp "/content/drive/My Drive/Colab Notebooks/Trabajo Final/val_df.csv" /content/
!cp "/content/drive/My Drive/Colab Notebooks/Trabajo Final/test_df.csv" /content/

In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/Trabajo Final/CheXpert-v1.0-small.zip" /content/
!unzip /content/CheXpert-v1.0-small.zip -d /content/

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/CheXpert-v1.0-small/train/patient52284/study1/view1_frontal.jpg  
  inflating: /content/__MACOSX/CheXpert-v1.0-small/train/patient52284/study1/._view1_frontal.jpg  
  inflating: /content/CheXpert-v1.0-small/train/patient53356/study1/view1_frontal.jpg  
  inflating: /content/__MACOSX/CheXpert-v1.0-small/train/patient53356/study1/._view1_frontal.jpg  
  inflating: /content/CheXpert-v1.0-small/train/patient14491/study1/view1_frontal.jpg  
  inflating: /content/__MACOSX/CheXpert-v1.0-small/train/patient14491/study1/._view1_frontal.jpg  
  inflating: /content/CheXpert-v1.0-small/train/patient37613/study1/view1_frontal.jpg  
  inflating: /content/__MACOSX/CheXpert-v1.0-small/train/patient37613/study1/._view1_frontal.jpg  
  inflating: /content/CheXpert-v1.0-small/train/patient03426/study1/view1_frontal.jpg  
  inflating: /content/__MACOSX/CheXpert-v1.0-small/train/patient03426/study1/._view1_frontal.jpg  
  in

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
import timm
import torch.nn as nn
import torch.optim as optim

class CheXpertDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform
        self.classes = [
            'Enlarged Cardiomediastinum', 'Cardiomegaly',
            'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation',
            'Pneumonia', 'Atelectasis', 'Pneumothorax',
            'Pleural Effusion', 'Pleural Other', 'Fracture'
        ]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.data.iloc[idx]['path_to_image'])
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        labels = torch.tensor(self.data.loc[idx, self.classes].values.astype(float))
        assert torch.all((labels >= 0) & (labels <= 1)), f"Label values out of range [0, 1] for index {idx}"
        return image, labels

# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Crear los datasets
image_dir = 'CheXpert-v1.0-small'
train_dataset = CheXpertDataset('train_df.csv', image_dir, transform=transform)
val_dataset = CheXpertDataset('val_df.csv', image_dir, transform=transform)
test_dataset = CheXpertDataset('test_df.csv', image_dir, transform=transform)

# Crear los dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


In [ ]:

# Cargar el modelo ViT preentrenado y ajustarlo para 12 clases
model = timm.create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 12)

# Congelar los parámetros del modelo base
for param in model.parameters():
    param.requires_grad = False

# Descongelar los parámetros de las últimas capas
for param in model.head.parameters():
    param.requires_grad = True

# Mover el modelo a GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Definir la función de pérdida y el optimizador
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Función para calcular el loss promedio
def calculate_loss(loader, model, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
    return total_loss / len(loader)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]', leave=True)
    for batch_idx, (data, target) in enumerate(train_pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)

        if loss.item() < 0:
            print(f"Warning: Negative loss detected in batch {batch_idx}. Loss value: {loss.item()}")
            print("Output:", output)
            print("Target:", target)
            raise ValueError("Negative loss detected")

        loss.backward()
        optimizer.step()

        train_pbar.set_postfix({'train_loss': f'{loss.item():.4f}'})

    # Validación
    model.eval()
    val_losses = []
    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]', leave=True)
        for data, target in val_pbar:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss = criterion(output, target)
            val_losses.append(val_loss.item())
            val_pbar.set_postfix({'val_loss': f'{val_loss.item():.4f}'})

    avg_val_loss = sum(val_losses) / len(val_losses)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Validation Loss: {avg_val_loss:.4f}')


Epoch 1/10 [Val]: 100%|██████████| 228/228 [01:01<00:00,  3.73it/s, val_loss=0.3136]


Epoch 1/10, Average Validation Loss: 0.3080


Epoch 2/10 [Val]: 100%|██████████| 228/228 [01:01<00:00,  3.71it/s, val_loss=0.3502]


Epoch 2/10, Average Validation Loss: 0.3093


Epoch 3/10 [Val]: 100%|██████████| 228/228 [01:01<00:00,  3.72it/s, val_loss=0.3237]


...

Epoch 8/10, Average Validation Loss: 0.3067


Epoch 9/10 [Val]: 100%|██████████| 228/228 [01:01<00:00,  3.73it/s, val_loss=0.3417]


Epoch 9/10, Average Validation Loss: 0.3198


Epoch 10/10 [Val]: 100%|██████████| 228/228 [01:01<00:00,  3.72it/s, val_loss=0.3550]

Epoch 10/10, Average Validation Loss: 0.3068


In [ ]:
from tqdm import tqdm

# Evaluación
test_loss = calculate_loss(test_loader, model, criterion)
print(f'Test Loss: {test_loss:.4f}')

# Calcular accuracy
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in tqdm(test_loader, desc="Evaluating"):
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = (output.sigmoid() > 0.5).float()
        correct += (pred == target).sum().item()
        total += target.numel()

accuracy = correct / total
print(f'Accuracy: {accuracy:.4f}')


Test Loss: 0.3086


Evaluating: 100%|██████████| 229/229 [01:00<00:00,  3.78it/s]

Accuracy: 0.8797


In [ ]:

# Calcular accuracy por patología
model.eval()
class_correct = [0] * 12
class_total = [0] * 12
with torch.no_grad():
    for data, target in tqdm(test_loader, desc="Calculating per-pathology accuracy"):
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = (output.sigmoid() > 0.5).float()

        for i in range(target.shape[0]):  # Itera sobre las imágenes en el batch
            for j in range(target.shape[1]):  # Itera sobre las patologías
                if target[i,j] == 0 or target[i,j] == 1: # Si la etiqueta no es -1
                  class_total[j] += 1
                  if pred[i,j] == target[i,j]:
                      class_correct[j] += 1


for i in range(12):
    accuracy = 100 * class_correct[i] / class_total[i] if class_total[i] > 0 else 0
    print(f"Accuracy of {train_dataset.classes[i]}: {accuracy:.2f}%")


Calculating per-pathology accuracy: 100%|██████████| 229/229 [01:07<00:00,  3.40it/s]

Accuracy of Enlarged Cardiomediastinum: 94.24%
Accuracy of Cardiomegaly: 89.48%
Accuracy of Lung Opacity: 60.85%
Accuracy of Lung Lesion: 96.62%
Accuracy of Edema: 77.76%
Accuracy of Consolidation: 94.30%
Accuracy of Pneumonia: 97.78%
Accuracy of Atelectasis: 84.65%
Accuracy of Pneumothorax: 93.37%
Accuracy of Pleural Effusion: 73.35%
Accuracy of Pleural Other: 98.68%
Accuracy of Fracture: 94.55%


In [ ]:
# Después del entrenamiento
torch.save(model.state_dict(), 'modelo_chexpert_params.pth')

In [ ]:
# Después del entrenamiento
torch.save(model.state_dict(), '/content/drive/My Drive/Diagnovision/model_img/best_models/modelo_chexpert_params.pth')


In [ ]:
# Para cargar los parámetros más tarde
modelo_nuevo = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=12)
modelo_nuevo.load_state_dict(torch.load('/content/drive/My Drive/Diagnovision/model_img/best_models/modelo_chexpert_params.pth'))
modelo_nuevo.eval()  # Poner el modelo en modo evaluación

In [ ]:
# Para cargar los parámetros más tarde y ejecutar una prueba con una imagen nueva
import torch
import timm
from PIL import Image
from torchvision import transforms

# Definir las transformaciones (las mismas que usaste para entrenar)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Cargar el modelo preentrenado
modelo_nuevo = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=12)
modelo_nuevo.load_state_dict(torch.load('/content/drive/My Drive/Diagnovision/model_img/best_models/modelo_chexpert_params.pth'))
modelo_nuevo.eval()  # Poner el modelo en modo evaluación

# Mover el modelo a GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo_nuevo = modelo_nuevo.to(device)


# Ruta de la imagen nueva
image_path = "/content/CheXpert-v1.0-small/train/patient40920/study1/view1_frontal.jpg" # Reemplaza con la ruta correcta a tu imagen

# Cargar y preprocesar la imagen
image = Image.open(image_path).convert('RGB')
image = transform(image).unsqueeze(0) # Agregar una dimensión para el batch
image = image.to(device)

# Hacer la predicción
with torch.no_grad():
  output = modelo_nuevo(image)
  probabilities = torch.sigmoid(output)

# Imprimir las probabilidades
print(probabilities)

# Obtener las predicciones (0 o 1)
predictions = (probabilities > 0.5).float()
predictions


<ipython-input-23-da8ec18dc312>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo_nuevo.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/Trabajo Fi

tensor([[0.0168, 0.0584, 0.4243, 0.0160, 0.2896, 0.0252, 0.0074, 0.2245, 0.0522,
         0.6251, 0.0024, 0.0075]], device='cuda:0')


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], device='cuda:0')

In [ ]:
# Hacer la predicción
with torch.no_grad():
  output = modelo_nuevo(image)
  probabilities = torch.sigmoid(output)

# Obtener las predicciones (0 o 1) y las probabilidades
predictions = (probabilities > 0.5).float()

# Imprimir las probabilidades con las patologías
classes = [
    'Enlarged Cardiomediastinum', 'Cardiomegaly',
    'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation',
    'Pneumonia', 'Atelectasis', 'Pneumothorax',
    'Pleural Effusion', 'Pleural Other', 'Fracture'
]

for i in range(len(classes)):
  print(f"{classes[i]}: Probability = {probabilities[0][i].item():.4f}, Prediction = {predictions[0][i].item()}")

Enlarged Cardiomediastinum: Probability = 0.0168, Prediction = 0.0
Cardiomegaly: Probability = 0.0584, Prediction = 0.0
Lung Opacity: Probability = 0.4243, Prediction = 0.0
Lung Lesion: Probability = 0.0160, Prediction = 0.0
Edema: Probability = 0.2896, Prediction = 0.0
Consolidation: Probability = 0.0252, Prediction = 0.0
Pneumonia: Probability = 0.0074, Prediction = 0.0
Atelectasis: Probability = 0.2245, Prediction = 0.0
Pneumothorax: Probability = 0.0522, Prediction = 0.0
Pleural Effusion: Probability = 0.6251, Prediction = 1.0
Pleural Other: Probability = 0.0024, Prediction = 0.0
Fracture: Probability = 0.0075, Prediction = 0.0


In [ ]:
# Obtener las 3 probabilidades más altas
top3_probabilities, top3_indices = torch.topk(probabilities, 3)

# Imprimir las 3 patologías con mayor probabilidad
for i in range(3):
    index = top3_indices[0][i].item()
    print(f"{classes[index]}: Probability = {top3_probabilities[0][i].item():.4f}, Prediction = {predictions[0][index].item()}")


Pleural Effusion: Probability = 0.6251, Prediction = 1.0
Lung Opacity: Probability = 0.4243, Prediction = 0.0
Edema: Probability = 0.2896, Prediction = 0.0
